In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time
from scipy.stats import rankdata

In [2]:
def lap_noise(data, epsilon, K, N, M):
    pri_data = data + np.random.laplace(loc = 0.0, scale = (K*(16*N-11)/N)/epsilon, size=data.shape)
    
    min = 10000
    
    for i in range(M):
        if pri_data[i] > 0 and pri_data[i] < min:
            min = pri_data[i]
    
    for i in range(M):
        if pri_data[i] < 0:
            pri_data[i] = min
    
    return pri_data

In [3]:
def total(n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    
    if h == 0:
        return 0
    else:
        return (4*((i-h/4)**2) + 2*((i+j-h/2)**2) + 4*((j-h/4)**2))/h

In [4]:
def f(x, y, theta):
    t = (x - math.sqrt(2)*math.cos(theta))**2 + (y - math.sin(theta))**2
    return math.sqrt(t)

def eDistance(x,y):
    if x != 0:
        roots = np.roots([4, 4*y, 8*x*x + y*y -4, -4*y, -y*y])
        #print(roots)
        theta = np.zeros(9)
        theta[0] = 0
        c = 1
        for i in range(4):
            if roots[i] >= -1 and roots[i] <= 1:
                theta[c] = math.asin(roots[i])
                if theta[c] >= 0:
                    theta[c+1] = math.pi - theta[c]
                else:
                    theta[c+1] = - math.pi - theta[c]
                c += 2
        #print(theta)
        
        distance = 1000000000
        for j in range(c):
            #print(f(x,y,theta[j]))
            if f(x,y,theta[j]) < distance:
                distance = f(x,y,theta[j])
                
    else:
        if y > 0:
            distance = math.fabs(1-y)
        else:
            distance = math.fabs(-1-y)
    return distance

In [5]:
def appx_SHD_total(cc,n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    T = total(n)
    
    if h <= cc:
        s = 2*(i-j)/cc
        t = 2*(i+j-cc/2)/cc
        d = eDistance(s,t)
        if T < cc:
            return -math.ceil((math.sqrt(2)-1)*(cc-h)/(2*math.sqrt(2)) + d*cc/8)
        else:
            return math.ceil((math.sqrt(2)-1)*(cc-h)/(2*math.sqrt(2)) + d*cc/8)-1
    else:
        s = 2*(i-j)/math.sqrt(h*cc)
        t = 2*(i+j-h/2)/math.sqrt(h*cc)
        d = eDistance(s,t)
        if T < cc:
            return -math.ceil(d*math.sqrt(h*cc)/8)
        else:
            return math.ceil(d*math.sqrt(h*cc)/8)-1

In [6]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_mec(shd, m, K, epsilon):
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * shd[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
        
        x = random_num(p)
        shd[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [7]:
for j in range(5):
    N = 150
    M = 5000
    H = np.zeros(M)
    I = np.zeros(M)
    J = np.zeros(M)
    
    n = np.zeros((M,10))
    
    for i in range(M-10):
        n[i][4] = np.random.binomial(2*N, 1/5)
        S = np.random.binomial(2*N-n[i][4], 1/4) #n7+n9+n10
        P = np.random.binomial(S, 1/2) #n7+n10
        n[i][6] = np.random.binomial(P, 1/2)
        n[i][9] = P - n[i][6]
        n[i][8] = S - P
        n[i][1] = np.random.binomial(2*N-n[i][4]-S, 1/3)
        T = np.random.binomial(2*N-n[i][4]-n[i][1]-S, 1/2) #n3+n4
        n[i][2] = np.random.binomial(T, 1/2)
        n[i][3] = T - n[i][2]
        R = 2*N-n[i][4]-n[i][1]-S-T
        Q = np.random.binomial(R, 1/2) #n6+n8
        n[i][5] = np.random.binomial(Q, 1/2)
        n[i][7] = Q - n[i][5]
        n[i][0] = R - Q
        
        H[i] = n[i][1]+n[i][2]+n[i][3] + 2*(n[i][4]+n[i][5]+n[i][6]+n[i][7]+n[i][8]+n[i][9])
        I[i] = n[i][3]+n[i][7]+n[i][8] + 2*n[i][9]
        J[i] = n[i][2]+n[i][5] + 2*n[i][6] + n[i][8]
    
    for i in range(M-10,M):
        n[i][4] = np.random.binomial(2*N, 1.5/5)
        S = np.random.binomial(2*N-n[i][4], 1/4) #n7+n9+n10
        P = np.random.binomial(S, 1/2) #n7+n10
        n[i][6] = np.random.binomial(P, 1.5/2)
        n[i][9] = P - n[i][6]
        n[i][8] = S - P
        n[i][1] = np.random.binomial(2*N-n[i][4]-S, 1.5/3)
        T = np.random.binomial(2*N-n[i][4]-n[i][1]-S, 1/2) #n3+n4
        n[i][2] = np.random.binomial(T, 1.5/2)
        n[i][3] = T - n[i][2]
        R = 2*N-n[i][4]-n[i][1]-S-T
        Q = np.random.binomial(R, 1/2) #n6+n8
        n[i][5] = np.random.binomial(Q, 1.5/2)
        n[i][7] = Q - n[i][5]
        n[i][0] = R - Q
        
        H[i] = n[i][1]+n[i][2]+n[i][3] + 2*(n[i][4]+n[i][5]+n[i][6]+n[i][7]+n[i][8]+n[i][9])
        I[i] = n[i][3]+n[i][7]+n[i][8] + 2*n[i][9]
        J[i] = n[i][2]+n[i][5] + 2*n[i][6] + n[i][8]
    
    stats = np.zeros(M)
    
    for i in range(M):
        if H[i] == 0:
            stats[i] = 0
        else:
            stats[i] = (2*((I[i]-J[i])**2))/H[i] + ((2*I[i] + 2*J[i] - H[i])**2)/H[i]
    
    shd = np.zeros(M)
    appx_shd = np.zeros(M)
    cc = 23.0
    epsilon = 3
    K = 3
    
    lap_start = time.time()
    lap_noise(stats, epsilon, K, N, M)
    lap_end = time.time()
    
    print("lap_time =",lap_end-lap_start,"[sec]")
    
    appx_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5]), int(n[i][6]), int(n[i][7]), int(n[i][8]), int(n[i][9])]
        appx_shd[i] = appx_SHD_total(cc,k)
    exp_mec(appx_shd, M, K, epsilon)
    appx_end = time.time()
    
    print("appx_time =",appx_end-appx_start,"[sec]")
    print("----------")

lap_time = 0.003866910934448242 [sec]


<ipython-input-4-93072744abec>:14: ComplexWarning: Casting complex values to real discards the imaginary part
  theta[c] = math.asin(roots[i])


appx_time = 0.4326510429382324 [sec]
----------
lap_time = 0.004553079605102539 [sec]
appx_time = 0.4326009750366211 [sec]
----------
lap_time = 0.0042209625244140625 [sec]
appx_time = 0.4362790584564209 [sec]
----------
lap_time = 0.004142045974731445 [sec]
appx_time = 0.4447510242462158 [sec]
----------
lap_time = 0.0040471553802490234 [sec]
appx_time = 0.4362819194793701 [sec]
----------
